In [1]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from python_module.pricing_model import BSMModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.5f}'.format

In [2]:
def compute_numeric_derivative(pricing_fun, base_params, param_names, h=None):
    """
    Numerically compute derivative(s) of pricing_fun at base_params.

    - pricing_fun: callable(**kwargs) -> scalar price
    - base_params: dict of parameters to pass to pricing_fun
    - param_names: str or (str, str)
        * If str -> first derivative w.r.t that parameter (central difference)
        * If (p, p) -> second derivative w.r.t p (second central difference)
        * If (p, q) with p != q -> mixed second partial d2/dp dq (central)
    - h: optional bump size (scalar or dict mapping param->h). If None, automatic h used.

    Returns numeric derivative (float).
    """
    import numpy as np

    def safe_eval(params):
        val = pricing_fun(**params)
        a = np.asarray(val)
        if a.size != 1:
            raise ValueError("pricing_fun must return a scalar-like value")
        return float(a.item())

    eps = np.finfo(float).eps
    # normalize param_names to tuple
    if isinstance(param_names, str):
        params = (param_names,)
    else:
        params = tuple(param_names)

    # helper to compute adaptive h for a single param
    def get_h_for(name, x):
        if h is None:
            return (eps ** (1/3)) * (abs(x) + 1.0)
        if isinstance(h, dict):
            return float(h.get(name, (eps ** (1/3)) * (abs(x) + 1.0)))
        return float(h)

    # single derivative (first)
    if len(params) == 1:
        p = params[0]
        x0 = float(base_params[p])
        hh = get_h_for(p, x0)
        p_plus = dict(base_params); p_plus[p] = x0 + hh
        p_minus = dict(base_params); p_minus[p] = x0 - hh
        f_plus = safe_eval(p_plus)
        f_minus = safe_eval(p_minus)
        return (f_plus - f_minus) / (2.0 * hh)

    # two params -> second derivative or mixed partial
    if len(params) == 2:
        p, q = params
        x_p = float(base_params[p])
        x_q = float(base_params[q])
        h_p = get_h_for(p, x_p)
        h_q = get_h_for(q, x_q)

        if p == q:
            # second derivative w.r.t same parameter
            p_plus = dict(base_params); p_plus[p] = x_p + h_p
            p_minus = dict(base_params); p_minus[p] = x_p - h_p
            f_plus = safe_eval(p_plus)
            f0 = safe_eval(base_params)
            f_minus = safe_eval(p_minus)
            return (f_plus - 2.0 * f0 + f_minus) / (h_p ** 2)
        else:
            # mixed partial: central 4-point formula
            pp = dict(base_params); pp[p] = x_p + h_p; pp[q] = x_q + h_q
            pm = dict(base_params); pm[p] = x_p + h_p; pm[q] = x_q - h_q
            mp = dict(base_params); mp[p] = x_p - h_p; mp[q] = x_q + h_q
            mm = dict(base_params); mm[p] = x_p - h_p; mm[q] = x_q - h_q
            f_pp = safe_eval(pp)
            f_pm = safe_eval(pm)
            f_mp = safe_eval(mp)
            f_mm = safe_eval(mm)
            return (f_pp - f_pm - f_mp + f_mm) / (4.0 * h_p * h_q)

    raise ValueError("param_names must be a string or a tuple/list of two strings")

In [3]:
# wrapper if your model returns a dict or complex object
def price_scalar(**kwargs):
    return BSMModel.compute_option_with_forward(**kwargs)  # must return scalar

base = dict(F=100, K=100, T=1, r=0, sigma=0.3, option_type='call', compute_greeks=False)

delta = compute_numeric_derivative(price_scalar, base, 'F')
gamma = compute_numeric_derivative(price_scalar, base, ('F','F'))
vega = compute_numeric_derivative(price_scalar, base, ('sigma'))
theta = compute_numeric_derivative(price_scalar, base, ('T'))
vanna = compute_numeric_derivative(price_scalar, base, ('F','sigma'))
volga = compute_numeric_derivative(price_scalar, base, ('sigma','sigma'))

In [4]:
delta, gamma, vega/100, theta/-250, vanna, volga

(0.5596176923562872,
 0.013149315164148868,
 0.39447933090164694,
 -0.02366875985292543,
 0.1972393877324846,
 -2.958673546784521)

In [5]:
base = dict(F=100, K=100, T=1, r=0, sigma=0.3, option_type='call', compute_greeks=True)
BSMModel.compute_option_with_forward(**base)

{'price': 11.923538474048499,
 'delta': 0.5596176923702425,
 'gamma': 0.013149311030262964,
 'vega': 0.39447933090788895,
 'theta': -0.023480912554041007,
 'vanna': 0.19723966545394447,
 'volga': -2.958594981809167}

In [6]:
BSMModel.compute_pnl_attribution(F_start=100, F_end=100, sigma_start=0.30, sigma_end=0.30, T_start=252/252, T_end=251/252, K=100, r=0, option_type='call')

{'total_pnl': -0.023504778274613614,
 'delta_pnl': 0.0,
 'gamma_pnl': 0.0,
 'theta_pnl': -0.02329455610519933,
 'vega_pnl': 0.0,
 'vanna_pnl': 0.0,
 'volga_pnl': -0.0,
 'unexplained_pnl': -0.00021022216941428418}

In [7]:
BSMModel.compute_pnl_attribution(F_start=100, F_end=100, sigma_start=0.30, sigma_end=0.31, T_start=252/252, T_end=252/252, K=100, r=0, option_type='call')

{'total_pnl': 0.3943297954008713,
 'delta_pnl': 0.0,
 'gamma_pnl': 0.0,
 'theta_pnl': 0.0,
 'vega_pnl': 0.3944793309078893,
 'vanna_pnl': 0.0,
 'volga_pnl': -0.00014792974909045862,
 'unexplained_pnl': -1.605757927514162e-06}

In [11]:
BSMModel.compute_pnl_attribution(F_start=100, F_end=105, sigma_start=0.30, sigma_end=0.35, T_start=252/252, T_end=251/252, K=100, r=0, option_type='call')

{'total_pnl': 4.924775558524523,
 'delta_pnl': 2.7980884618512123,
 'gamma_pnl': 0.16436638787828706,
 'theta_pnl': -0.02329455610519933,
 'vega_pnl': 1.9723966545394442,
 'vanna_pnl': 0.049309916363486105,
 'volga_pnl': -0.0036982437272614567,
 'unexplained_pnl': -0.03239306227544603}